In [2]:
import json
import httpx
import dotenv
import os

dotenv.load_dotenv("../config/.env.development")


True

**Fetching Access Token**

In [ ]:
url = os.environ["SERVER_URL"] + "/authentication"
body = {
    "username": os.environ["SERVER_USERNAME"],
    "password": os.environ["SERVER_PASSWORD"]
}

auth_response = httpx.post(url=url, json=body)
assert str(auth_response.status_code).startswith("2"), auth_response
print(json.dumps(auth_response.json(), indent=4))
access_token = auth_response.json()['access_token']

**Creating a Sensor**

In [ ]:
url = os.environ["SERVER_URL"] + "/sensors"
headers = {
    "authorization": f"Bearer {access_token}"
}
body = {
    "sensor_name": "moritz-macbook-pro-14",
    "network_identifier": "81bf7042-e20f-4a97-ac44-c15853e3618f",
    "configuration": {
        "version": "0.1.0-alpha.5",
        "active_components": {
            "calibration_procedures": False,
            "mqtt_data_sending": True,
            "heated_enclosure_communication": False,
            "pump_speed_monitoring": False
        },
        "hardware": {
            "pumped_litres_per_round": 0.003,
            "inner_tube_diameter_millimiters": 4
        },
        "measurement": {
            "timing": {
                "seconds_per_measurement_interval": 120,
                "seconds_per_measurement": 10
            },
            "pumped_litres_per_minute": 4,
            "air_inlets": [
                { "valve_number": 1, "direction": 300, "tube_length": 50 },
                { "valve_number": 2, "direction": 50, "tube_length": 50 }
            ]
        },
        "calibration": {
            "hours_between_calibrations": 25,
            "gases": [
                { "valve_number": 3, "concentration": 400 },
                { "valve_number": 4, "concentration": 800 }
            ],
            "flushing": {
                "seconds": 300,
                "pumped_litres_per_minute": 0.5
            },
            "sampling": {
                "pumped_litres_per_minute": 0.5,
                "sample_count": 20,
                "seconds_per_sample": 300
            },
            "cleaning": {
                "seconds": 300,
                "pumped_litres_per_minute": 0.5
            }
        },
        "heated_enclosure": {
            "target_temperature": 40,
            "allowed_deviation": 3
        }
    },
}

sensor_creation_response = httpx.post(url=url, headers=headers, json=body)
assert str(sensor_creation_response.status_code).startswith("2"), sensor_creation_response
print(json.dumps(sensor_creation_response.json(), indent=4))

**Store list of sensor identifiers**

In [ ]:
MQTT_SENSOR_IDENTIFIERS = {
    "tum-esm-midcost-raspi-1": "919ac396-de7d-4dda-8224-564739e0ff1b",
    "moritz-macbook-pro-14": "e04a6cb9-88c5-43b4-887d-72615d15d1e3",
}

**Sending data**

In [3]:
import sys
import time
import dotenv

sys.path.append("..")
dotenv.load_dotenv("../config/.env.development")

from src import custom_types, utils

utils.SendingMQTTClient.init_archiving_loop_process()
utils.SendingMQTTClient.init_sending_loop_process()
utils.SendingMQTTClient.check_errors()

config = utils.ConfigInterface.read()
sending_mqtt_client = utils.SendingMQTTClient()
sending_mqtt_client.enqueue_message(
    config,
    custom_types.MQTTLogMessageBody(
        severity="info",
        revision=config.revision,
        timestamp=time.time(),
        subject="Hello, world!",
        details="no one will read the anyways"
    )
)
sending_mqtt_client.enqueue_message(
    config,
    custom_types.MQTTDataMessageBody(
        revision=config.revision,
        timestamp=time.time(),
        value=custom_types.MQTTCO2Data(
            variant="co2",
            data=custom_types.CO2SensorData(
                raw=400,
                compensated=500,
                filtered=600
            )
        )
    )
)

time.sleep(10)
utils.SendingMQTTClient.check_errors()
